关于数据读取的问题
一般来说PyTorch中深度学习训练的流程是这样的： 1. 创建Dateset 2. Dataset传递给DataLoader 3. DataLoader迭代产生训练数据提供给模型
代码如下
```python
# 创建Dateset(可以自定义)
    dataset = face_dataset # Dataset部分自定义过的face_dataset
# Dataset传递给DataLoader
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=64,shuffle=False,num_workers=8)
# DataLoader迭代产生训练数据提供给模型
    for i in range(epoch):
        for index,(img,label) in enumerate(dataloader):
            pass
```
其中 `for index,(img,label) in enumerate(dataloader):` 也可以使用 iter() 方法替换，比如 `for img,label in iter(dataloader):` 
 dataloader 本质上是一个可迭代对象，可以使用 iter() 进行访问，采用 iter(dataloader) 返回的是一个迭代器，然后可以使用 next() 访问。
也可以使用 enumerate(dataloader) 的形式访问。
[Pytorch中iter(dataloader)的使用](https://blog.csdn.net/weixin_44533869/article/details/110856518)这篇文章中介绍了这两种方法及其使用，但是在使用 enumerate(dataloader) 时没有加 index 导致 img 和 label 的位置对换了，实测加上 index 后无误，即示例代码的样子。


In [3]:
#搭建神经网络的几种方法   1、2是感觉比较好用的两种方法,2.0是第2种方法当只有一个模块的情况下的简化形式，也比较方便。
import torch
import torch.nn.functional as F
from collections import OrderedDict

#1、继承Module类
class Net1(torch.nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, 3, 1, 1)
        self.dense1 = torch.nn.Linear(32 * 3 * 3, 128)
        self.dense2 = torch.nn.Linear(128, 10)
 
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv(x)), 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        return x
 
print("Method 1:")
model1 = Net1()
print(model1)
print('这种方法比较常用，早期的教程通常就是使用这种方法。')

#2、Sequential类
class Net2(torch.nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, 3, 1, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(32 * 3 * 3, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 10)
        )
 
    def forward(self, x):
        conv_out = self.conv1(x)
        res = conv_out.view(conv_out.size(0), -1)
        out = self.dense(res)
        return out
 
print("Method 2:")
model2 = Net2()
print(model2)
print('这种方法利用torch.nn.Sequential()容器进行快速搭建，模型的各层被顺序添加到容器中。缺点是每层的编号是默认的阿拉伯数字，不易区分。')

#2.0 2的简化情况
print("Method 2.0:")
net0 = torch.nn.Sequential(torch.nn.Linear(20, 256), torch.nn.ReLU(), torch.nn.Linear(256, 10))
print(net0)

#3、2的改进（自己用的话感觉用不到这个）
class Net3(torch.nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        self.conv = torch.nn.Sequential()
        self.conv.add_module("conv1",torch.nn.Conv2d(3, 32, 3, 1, 1))
        self.conv.add_module("relu1",torch.nn.ReLU())
        self.conv.add_module("pool1",torch.nn.MaxPool2d(2))
        self.dense = torch.nn.Sequential()
        self.dense.add_module("dense1",torch.nn.Linear(32 * 3 * 3, 128))
        self.dense.add_module("relu2",torch.nn.ReLU())
        self.dense.add_module("dense2",torch.nn.Linear(128, 10))
 
    def forward(self, x):
        conv_out = self.conv1(x)
        res = conv_out.view(conv_out.size(0), -1)
        out = self.dense(res)
        return out
 
print("Method 3:")
model3 = Net3()
print(model3)
print("这种方法是对第二种方法的改进：通过add_module()添加每一层，并且为每一层增加了一个单独的名字。")

#4、3的改写
class Net4(torch.nn.Module):
    def __init__(self):
        super(Net4, self).__init__()
        self.conv = torch.nn.Sequential(
            OrderedDict(
                [
                    ("conv1", torch.nn.Conv2d(3, 32, 3, 1, 1)),
                    ("relu1", torch.nn.ReLU()),
                    ("pool", torch.nn.MaxPool2d(2))
                ]
            ))
 
        self.dense = torch.nn.Sequential(
            OrderedDict([
                ("dense1", torch.nn.Linear(32 * 3 * 3, 128)),
                ("relu2", torch.nn.ReLU()),
                ("dense2", torch.nn.Linear(128, 10))
            ])
        )
 
    def forward(self, x):
        conv_out = self.conv1(x)
        res = conv_out.view(conv_out.size(0), -1)
        out = self.dense(res)
        return out
 
print("Method 4:")
model4 = Net4()
print(model4)
print("第三种方法的另外一种写法，通过字典的形式添加每一层，并且设置单独的层名称。")

Method 1:
Net1(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dense1): Linear(in_features=288, out_features=128, bias=True)
  (dense2): Linear(in_features=128, out_features=10, bias=True)
)
Method 2:
Net2(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=288, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)
Method 2.0:
Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


对于训练过程，基本的格式就是 嵌套循环 + 遍历一次数据 + 打印loss

1、嵌套循环
```
for epoch in range(num_epochs):
    for X, y in data_iter:#这里还可以使用enumerate方法来访问dataloader
```

2、遍历一次数据
主要分为3个部分
（1）、通过调用net(X)生成预测并计算损失loss（前向传播）。
```
loss = loss_func(net(x), y)
```
（2）、通过进行反向传播来计算梯度。
```
optimizer.zero_grad()
loss.backward()
```
（3）、通过调用优化器来更新模型参数。
```
optimizer.step()
```

3、打印loss
![打印loss](picture\打印loss.png)

In [1]:
#简洁实现

import random

import torch

## Python3 range() 函数返回的是一个可迭代对象（类型是对象），而不是列表类型， 所以打印的时候不会打印列表。
## Python3 list() 函数是对象迭代器，可以把range()返回的可迭代对象转为一个列表，返回的变量类型为列表。


## 人造数据集
def create_data(w, b, nums_example):
    X = torch.normal(0, 1, (nums_example, len(w)))
    y = torch.matmul(X, w) + b
    print("y_shape:", y.shape)
    y += torch.normal(0, 0.01, y.shape)  # 加入噪声
    return X, y.reshape(-1, 1)  # y从行向量转为列向量


true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = create_data(true_w, true_b, 1000)


## 读数据集
def read_data(batch_size, features, lables):
    nums_example = len(features)
    indices = list(range(nums_example))  # 生成0-999的元组，然后将range()返回的可迭代对象转为一个列表
    random.shuffle(indices)  # 将序列的所有元素随机排序。
    for i in range(0, nums_example, batch_size):  # range(start, stop, step)
        index_tensor = torch.tensor(indices[i: min(i + batch_size, nums_example)])
        yield features[index_tensor], lables[index_tensor]  # 通过索引访问向量


batch_size = 10
for X, y in read_data(batch_size, features, labels):
    print("X:", X, "\ny", y)
    break;

##初始化参数
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


# 定义模型
def net(X, w, b):
    return torch.matmul(X, w) + b


# 定义损失函数
def loss(y_hat, y):
    # print("y_hat_shape:",y_hat.shape,"\ny_shape:",y.shape)
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2  # 这里为什么要加 y_hat_shape: torch.Size([10, 1])  y_shape: torch.Size([10])


# 定义优化算法
def sgd(params, batch_size, lr):
    with torch.no_grad():  # with torch.no_grad() 则主要是用于停止autograd模块的工作，
        for param in params:
            param -= lr * param.grad / batch_size  ##  这里用param = param - lr * param.grad / batch_size会导致导数丢失， zero_()函数报错
            param.grad.zero_()  ## 导数如果丢失了，会报错‘NoneType’ object has no attribute ‘zero_’


# 训练模型
lr = 0.03
num_epochs = 3

for epoch in range(0, num_epochs):
    for X, y in read_data(batch_size, features, labels):
        f = loss(net(X, w, b), y)
        # 因为`f`形状是(`batch_size`, 1)，而不是一个标量。`f`中的所有元素被加到一起，
        # 并以此计算关于[`w`, `b`]的梯度
        f.sum().backward()
        sgd([w, b], batch_size, lr)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print("w {0} \nb {1} \nloss {2:f}".format(w, b, float(train_l.mean())))

print("w误差 ", true_w - w, "\nb误差 ", true_b - b)

y_shape: torch.Size([1000])
X: tensor([[-1.6564, -0.7528],
        [-0.4417, -0.4678],
        [-0.5204, -0.2620],
        [-0.9958,  2.4005],
        [-0.1910, -0.6708],
        [ 1.4716, -2.0913],
        [ 0.1304,  1.8677],
        [ 0.7979, -1.5860],
        [ 0.4219, -0.5894],
        [-0.3467,  1.3199]]) 
y tensor([[ 3.4553],
        [ 4.9128],
        [ 4.0512],
        [-5.9535],
        [ 6.1058],
        [14.2364],
        [-1.8966],
        [11.2016],
        [ 7.0429],
        [-0.9917]])
w tensor([[ 1.9383],
        [-3.2635]], requires_grad=True) 
b tensor([4.0289], requires_grad=True) 
loss 0.027315
w tensor([[ 1.9986],
        [-3.3943]], requires_grad=True) 
b tensor([4.1929], requires_grad=True) 
loss 0.000096
w tensor([[ 2.0000],
        [-3.3995]], requires_grad=True) 
b tensor([4.1995], requires_grad=True) 
loss 0.000052
w误差  tensor([[-2.8849e-05, -5.4000e+00],
        [ 5.3995e+00, -5.3883e-04]], grad_fn=<SubBackward0>) 
b误差  tensor([0.0005], grad_fn=<RsubBackward